In [1]:
import pandas as pd
import difflib

In [13]:
county_col = pd.read_csv('/home/shumway743/cis5500_group/cost_of_living/county_col.csv')
county_col = county_col[county_col.Family == '1p0c'][['State abv.','County','Total.1']]
county_col = county_col.rename({'State abv.':'state', 'County':'area_name','Total.1':'annual_cost'},axis=1)
county_col['cost_index'] = county_col.annual_cost / county_col.annual_cost.mean()
county_col['area_name_lower'] = county_col.area_name.str.lower()
county_col['type'] = 'county'
county_col.head()

,state,area_name,annual_cost,cost_index,area_name_lower,type
0,AL,Autauga County,39254.0,1.071717,autauga county,county
10,AL,Baldwin County,39579.0,1.080590,baldwin county,county
20,AL,Barbour County,34424.0,0.939848,barbour county,county
30,AL,Bibb County,38572.0,1.053097,bibb county,county
40,AL,Blount County,37547.0,1.025112,blount county,county


In [11]:
metro_col = pd.read_csv('/home/shumway743/cis5500_group/cost_of_living/metro_col.csv')
metro_col = metro_col[metro_col.Family == '1p0c'][['State abv.','Areaname','Total.1']]
metro_col = metro_col.rename({'State abv.':'state', 'Areaname':'area_name','Total.1':'annual_cost'},axis=1)
metro_col['cost_index'] = metro_col.annual_cost / metro_col.annual_cost.mean()
metro_col['type'] = 'metro'
metro_col.head()

,state,area_name,annual_cost,cost_index,type
0,TX,"Abilene, TX MSA",35905.0,0.953405,metro
10,OH,"Akron, OH MSA",31778.0,0.843819,metro
20,GA,"Albany, GA MSA",35684.0,0.947537,metro
30,OR,"Albany, OR MSA",40461.0,1.074383,metro
40,NY,"Albany-Schenectady-Troy, NY MSA",40629.0,1.078844,metro


In [17]:
census = pd.read_csv('/home/shumway743/cis5500_group/census/acs_2021.csv')

census_county = census[census.SUMLEVEL.isin({50})]
census_county['area_name'] = census_county.NAME.apply(lambda x: x.split(', ')[0].lower())
census_city = census[census.SUMLEVEL.isin({160})]
census_metro = census[census.SUMLEVEL.isin({310})]
census_metro['state_code'] = census_metro.NAME.apply(lambda x: x.split(', ')[1].split(' ')[0])
census_metro['area_name'] = census_metro.NAME.apply(lambda x: x.split(', ')[0])
census_county.head()

/tmp/ipykernel_4216/3693547385.py:1: DtypeWarning: Columns (5,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  census = pd.read_csv('/home/shumway743/cis5500_group/census/acs_2021.csv')
/tmp/ipykernel_4216/3693547385.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_county['area_name'] = census_county.NAME.apply(lambda x: x.split(', ')[0].lower())
/tmp/ipykernel_4216/3693547385.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_metro['state_code'] = census_

,Unnamed: 0,GEO_ID,FILEID,STUSAB,SUMLEVEL,COMPONENT,US,REGION,DIVISION,STATE,...,med_inc_public_administration,median_commute,high_school,associates,bachelors,masters,doctorate,unemployment_rate,pct_minority,area_name
601,601,0500000US01001,ACSSF,AL,50,00,NaN,NaN,NaN,1.0,...,59714,27,0.895845,0.372015,0.281315,0.117055,0.011713,0.027296,0.248699,autauga county
602,602,0500000US01003,ACSSF,AL,50,00,NaN,NaN,NaN,1.0,...,53004,27,0.910142,0.419232,0.324503,0.118430,0.013829,0.036685,0.154523,baldwin county
603,603,0500000US01005,ACSSF,AL,50,00,NaN,NaN,NaN,1.0,...,36771,22,0.756710,0.184940,0.111531,0.044179,0.006168,0.086242,0.544915,barbour county
604,604,0500000US01007,ACSSF,AL,50,00,NaN,NaN,NaN,1.0,...,55703,37,0.805381,0.186461,0.119138,0.039671,0.004235,0.097068,0.240585,bibb county
605,605,0500000US01009,ACSSF,AL,50,00,NaN,NaN,NaN,1.0,...,56602,37,0.836481,0.270532,0.149036,0.056015,0.004426,0.060130,0.075487,blount county


In [18]:
jdf = census_county.merge(county_col, left_on=['area_name','STUSAB'], right_on=['area_name_lower','state'], how='inner')
county_out = jdf[['GEO_ID','annual_cost','cost_index']]
county_out.count()

GEO_ID         3138
annual_cost    3138
cost_index     3138
dtype: int64

In [36]:
county_out.to_csv('output/county_cost_of_living.csv')

In [19]:
def get_match_metro(x):
    m = difflib.get_close_matches(x['clean_name'], census_metro[x['state'] == census_metro['state_code']].clean_name,n=1, cutoff=.90)
    return m[0] if len(m) else None




In [24]:
drop_words = {'cdp', 'city', 'village', 'town', 'borough', 'msa'}

# words = {}
# def word_count(x):
#     s = re.sub("[^\w' ]", "", x.lower()).split()
#     for w in s:
#         words[w] = words.get(w,0) + 1
# census_city.area_name.apply(word_count)
# city.area_name.apply(word_count)
# census_metro.area_name.apply(word_count)


# drop_words = set([x[0] for x in list(filter(lambda x: x[1] > 1000, words.items()))])
# print(drop_words)

def remove_words(x):
    out = x.split(',')[0]
    for w in list(drop_words):
        out = out.lower().replace(w,'')
    return out

In [27]:
# census_city['clean_name'] = census_city.area_name.apply(remove_words)
census_metro['clean_name'] = census_metro.area_name.apply(remove_words)
metro_col['clean_name'] = metro_col.area_name.apply(remove_words)

metro_col.clean_name = metro_col.clean_name.str.replace('[^a-zA-Z]', '')
census_metro.clean_name = census_metro.clean_name.str.replace('[^a-zA-Z]', '')
# census_city.clean_name = census_city.clean_name.str.replace('[^a-zA-Z]', '')
metro_col.head()
census_metro[['NAME','clean_name']].head()

/tmp/ipykernel_4216/932779767.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  census_metro['clean_name'] = census_metro.area_name.apply(remove_words)
/tmp/ipykernel_4216/932779767.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  metro_col.clean_name = metro_col.clean_name.str.replace('[^a-zA-Z]', '')
/tmp/ipykernel_4216/932779767.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  census_metro.clean_name = census_metro.clean_name.str.replace('[^a-zA-Z]', '')
/tmp/ipykernel_4216/932779767.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

,NAME,clean_name
121125,"Aberdeen, SD Micro Area",aberdeen
121126,"Aberdeen, WA Micro Area",aberdeen
121127,"Abilene, TX Metro Area",abilene
121128,"Ada, OK Micro Area",ada
121129,"Adrian, MI Micro Area",adrian


In [29]:
metro_col['match'] = metro_col.apply(get_match_metro, axis=1)
metro_col['match_score'] = metro_col.apply(lambda x: difflib.SequenceMatcher(None, x['clean_name'].lower(), x['match'].lower()).ratio() if x['clean_name'] and x['match'] else None ,axis=1)
metro_col.head()

,state,area_name,annual_cost,cost_index,type,clean_name,match,match_score
0,TX,"Abilene, TX MSA",35905.0,0.953405,metro,abilene,abilene,1.0
10,OH,"Akron, OH MSA",31778.0,0.843819,metro,akron,akron,1.0
20,GA,"Albany, GA MSA",35684.0,0.947537,metro,albany,albany,1.0
30,OR,"Albany, OR MSA",40461.0,1.074383,metro,albany,None,NaN
40,NY,"Albany-Schenectady-Troy, NY MSA",40629.0,1.078844,metro,albanyschenectadytroy,albanyschenectadytroy,1.0


In [31]:
metro_col[metro_col.match_score == 1].count()

state          285
area_name      285
annual_cost    285
cost_index     285
type           285
clean_name     285
match          285
match_score    285
dtype: int64

In [34]:
metro_min = metro_col
metro_out = census_metro.drop('area_name',axis=1).merge(metro_min, left_on=['state_code','clean_name'],right_on=['state','clean_name'],how='inner')
#todo: find best match
metro_out = metro_out[['GEO_ID','annual_cost','cost_index','NAME','area_name']]

metro_out['match_score'] = metro_out.apply(lambda x: difflib.SequenceMatcher(None, x['NAME'].lower(), x['area_name'].lower()).ratio() if x['NAME'] and x['area_name'] else None ,axis=1)
metro_out = metro_out.sort_values('match_score',ascending=False).drop_duplicates('GEO_ID')
metro_out.head()

,GEO_ID,annual_cost,cost_index,NAME,area_name,match_score
189,310M600US34980,41993.0,1.115063,"Nashville-Davidson--Murfreesboro--Franklin, TN...","Nashville-Davidson--Murfreesboro--Franklin, TN...",0.934426
161,310M600US30780,36297.0,0.963814,"Little Rock-North Little Rock-Conway, AR Metro...","Little Rock-North Little Rock-Conway, AR HUD M...",0.927273
68,310M600US19660,35435.0,0.940925,"Deltona-Daytona Beach-Ormond Beach, FL Metro Area","Deltona-Daytona Beach-Ormond Beach, FL HUD Met...",0.924528
62,310M600US18880,41712.0,1.107602,"Crestview-Fort Walton Beach-Destin, FL Metro Area","Crestview-Fort Walton Beach-Destin, FL HUD Met...",0.924528
120,310M600US26420,37357.0,0.991961,"Houston-The Woodlands-Sugar Land, TX Metro Area","Houston-The Woodlands-Sugar Land, TX HUD Metro...",0.921569


In [35]:
metro_out.to_csv('output/metro_cost_of_living.csv')